# ルートを評価するために解くLPについて
- ルートが1つ与えられた後、そのルートを評価するためにLPを解く(ルートが実行可能であるとは限らない)

<!--- 各制約の違反度を変数とし、それらに重みをかけて足し合わせた関数の最小化問題とする-->
- 車両が各顧客へ到着する時刻を変数とし、その合計を最小化する問題とする
- 制約は、容量制約と時間枠制約とする
    - 容量制約は、ある区間における2つの関数の積分値(面積)の大小を比較するという形で表す(車両の積荷の量を表す区分線形関数の\[x0, xn\]までの積分値と最大容量を表す線形関数の\[x0, xn\]までの積分値)
    - 時間枠制約は、通常のVRPの定式化と同様に表す


# 前準備

## instances.pyからインスタンスを得る

In [1]:
import instances
Customers = instances.Customers
Points = instances.Points
C, F = instances.C, instances.F
tour = instances.tour

In [2]:
instance_name = "C" + str(len(Customers)-1)

In [3]:
tour[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [4]:
tour[-10:]

[1, 2, 3, 4, 5, 6, 7, 8, 9, 'depot']

In [5]:
Points

[(6, 7), (0, 1), (4, 5)]

# 問題を解く
2つの手法に対し、計算時間を比較

## 入力する行列、ベクトルの作成

In [6]:
def distance(x1, y1, x2, y2):
    return ((x2-x1)**2+(y2-y1)**2)**(0.5)

In [7]:
#def make_preceding_constr(tour, Customers, As, Ac_leq, b_leq):
def make_preceding_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq):
    # ルート内の顧客の順序に関する制約
    for index, i in enumerate(tour):
        try:
            i_next = tour[index+1]
        except:
            break
        As.append([1 if target==i else -1 if target==i_next else 0 for target in tour])
        Ac_leq.append([1 if target==i else 0 for target in tour])
        b_leq.append(-distance(Customers[i].x, Customers[i].y, Customers[i_next].x, Customers[i_next].y))
        ## 以下はすべての行列のサイズを合わせるために行を追加している
        Ar_leq.append([0 for target in tour])
        Ap.append([0 for target in tour])
        Aq.append([0 for target in tour])
    #return As, Ac_leq, b_leq
    return As, Ac_leq, Ar_leq, Ap, Aq, b_leq

In [8]:
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2 \
    = [], [], [], [], [], [], [], [], [], [1 for _ in range(len(tour))], [1 for _ in range(len(tour))]
As, Ac_leq, Ar_leq, Ap, Aq, b_leq = make_preceding_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq)

In [9]:
b_leq

[-2.23606797749979,
 -2.23606797749979,
 -3.1622776601683795,
 -9.899494936611665,
 -10.63014581273465,
 -11.313708498984761,
 -11.40175425099138,
 -8.54400374531753,
 -3.605551275463989,
 -10.63014581273465]

In [10]:
#def make_tw_constr(tour, Customers, As, Ap, b_leq):
def make_tw_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq):
    # 時間枠制約
    for index, i in enumerate(tour):
        #As.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        #Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        As.append([-1 if i==target else 0 for target in tour])
        Ap.append([-1 if i==target else 0 for target in tour])
        b_leq.append(-Customers[i].e)
        #As.append([1 if i==target else 0 for index_, target in enumerate(tour)])
        #Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        As.append([1 if i==target else 0 for target in tour])
        Ap.append([-1 if i==target else 0 for target in tour])
        b_leq.append(Customers[i].l)
        ## 以下はすべての行列のサイズを合わせるために行を追加している
        for _ in range(2):
            Ac_leq.append([0 for target in tour])
            Ar_leq.append([0 for target in tour])
            Aq.append([0 for target in tour])
    #return As, Ap, b_leq
    return As, Ac_leq, Ar_leq, Ap, Aq, b_leq

In [11]:
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2 \
    = [], [], [], [], [], [], [], [], [], [1 for _ in range(len(tour))], [1 for _ in range(len(tour))]
As, Ac_leq, Ar_leq, Ap, Aq, b_leq = make_tw_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq)

In [12]:
b_leq

[-1,
 3,
 -4,
 6,
 -8,
 9,
 -10,
 11,
 -12,
 16,
 -17,
 17,
 -18,
 21,
 -22,
 25,
 -27,
 28,
 -29,
 33,
 0,
 0]

In [13]:
#def make_nonnega_constr(tour, As, Ac_leq, Ar_leq, Ap, Aq, b_leq):
def make_nonnega_constr(tour, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq):
    # 変数の非負制約
    for index, i in enumerate(tour):
        As.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ac_leq.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ar_leq.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Ap.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        Aq.append([-1 if i==target else 0 for index_, target in enumerate(tour)])
        b_leq.append(0)
    return As, Ac_leq, Ar_leq, Ap, Aq, b_leq

In [15]:
#def make_time_limit_constr(tour, Customers, Points, As, Aq, b_leq):
def make_time_limit_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq):
    # ピックアップからデリバリーまでの時間制限に関する制約
    for index, pair in enumerate(Points):
        p = pair[0]
        d = pair[1]
        As.append([-1 if target==p else 1 if target==d else 0 for target in tour])
        Aq.append([-1 if target==p else 0 for target in tour])
        b_leq.append(Customers[p].t)
        ## 以下はすべての行列のサイズを合わせるために行を追加している
        Ac_leq.append([0 for target in tour])
        Ar_leq.append([0 for target in tour])
        Ap.append([0 for target in tour])
    #return As, Aq, b_leq
    return As, Ac_leq, Ar_leq, Ap, Aq, b_leq

In [16]:
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2 \
    = [], [], [], [], [], [], [], [], [], [1 for _ in range(len(tour))], [1 for _ in range(len(tour))]
As, Ac_leq, Ar_leq, Ap, Aq, b_leq = make_time_limit_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq)

In [17]:
As

[[0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0],
 [-1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -1, 1, 0, 0, 0, 0, 0]]

In [18]:
Aq

[[0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0],
 [-1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0]]

In [19]:
b_leq

[65, 51, 38]

In [20]:
Points

[(6, 7), (0, 1), (4, 5)]

In [21]:
#def make_charge_constr(tour, Ac_eq, Ac_leq, Ar_eq, Ar_leq, b_eq, b_leq, F, C):
def make_charge_constr(tour, Customers, Points, As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, F, C):
    # 自動車の燃料を補充することに関する制約
    for i in tour:
        Ac_leq.append([1 if i==target else 0 for target in tour])
        Ar_leq.append([1 if i==target else 0 for target in tour])
        b_leq.append(C)
        ## 以下はすべての行列のサイズを合わせるために行を追加している
        As.append([0 for target in tour])
        Ap.append([0 for target in tour])
        Aq.append([0 for target in tour])
        if i==0:
            Ar_eq.append([1] + [0 for j in range(len(tour)-1)])
            b_eq.append(F)
            ## 以下はすべての行列のサイズを合わせるために行を追加している
            Ac_eq.append([0 for target in tour])
            """elif i=="depot":
            continue"""
        else:
            if i=="depot":
                i_prev = tour[-1]
            else:
                i_prev = i-1
            Ac_eq.append([-1 if i_prev==target else 0 for target in tour])
            Ar_eq.append([1 if i==target else -1 if i_prev==target else 0 for target in tour])
            b_eq.append(distance(Customers[i].x, Customers[i].y, Customers[i_prev].x, Customers[i_prev].y))
    #return Ac_eq, Ac_leq, Ar_eq, Ar_leq, b_eq, b_leq
    return As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq

In [22]:
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2 \
    = [], [], [], [], [], [], [], [], [], [1 for _ in range(len(tour))], [1 for _ in range(len(tour))]
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq = make_charge_constr(tour, Customers, Points, As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, F, C)

In [23]:
def make_inputs(tour, Customers, Points, F, C):
    import numpy as np
    As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2 \
        = [], [], [], [], [], [], [], [], [], [1 for _ in range(len(tour))], [1 for _ in range(len(tour))]
    
    # ルート内の顧客の順序に関する制約
    ##As, Ac_leq, b_leq = make_preceding_constr(tour, Customers, As, Ac_leq, b_leq)
    As, Ac_leq, Ar_leq, Ap, Aq, b_leq = make_preceding_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq)
    print("Preceding constraints are done.")
    
    # 時間枠制約
    ##As, Ap, b_leq = make_tw_constr(tour, Customers, As, Ap, b_leq)
    As, Ac_leq, Ar_leq, Ap, Aq, b_leq = make_tw_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq)
    print("Time-window constarints are done.")
    
    # 非負制約
    ##As, Ac_leq, Ar_leq, Ap, Aq, b_eq = make_nonnega_constr(tour, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq)
    ##print("Non-negative constarints are done.")
    
    # ピックアップからデリバリーまでの時間制限に関する制約
    #As, Aq, b_leq = time_limit_constr(tour, Customers, Points, As, Aq, b_leq)
    As, Ac_leq, Ar_leq, Ap, Aq, b_leq = make_time_limit_constr(tour, Customers, Points, As, Ac_leq, Ar_leq, Ap, Aq, b_leq)
    print("Pick-up and delivery constarints are done.")
    
    # 自動車の燃料を補充することに関する制約
    #Ac_eq, Ac_leq, Ar_eq, Ar_leq, b_eq, b_leq = charge_constr(tour, Ac_eq, Ac_leq, Ar_eq, Ar_leq, b_eq, b_leq, F, C)
    As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq = make_charge_constr(tour, Customers, Points, As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, F, C)
    print("Charge constarints are done.")
    
    # ndarrayに変換
    As = np.array(As)
    Ac_eq = np.array(Ac_eq)
    Ac_leq = np.array(Ac_leq)
    Ar_eq = np.array(Ar_eq)
    Ar_leq = np.array(Ar_leq)
    Ap = np.array(Ap)
    Aq = np.array(Aq)
    b_eq = np.array(b_eq)
    b_leq = np.array(b_leq)
    c1 = np.array(c1)
    c2 = np.array(c2)
    return As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2

## 主問題を解く関数の定義
与えられた定数を元にLPのモデルを作成した上でそれを解き、最適解を返す関数

In [24]:
def solve_primal(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2, instance_name, num_vars):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    # 定数を設定　←　入力として与えられる
    # 変数を設定
    """
    s_i : 顧客iへ車両が到着する時刻を表す変数
    r_i : 顧客 i における車両の燃料補充時間を表す変数
    c_i : 顧客 i に到着した時の燃料残量を表す変数
    p_i : 顧客iの時間枠の違反度を表す変数
    q_i : 顧客 i の荷物を配達するまでの制限時間に関する違反量を表す変数
    """
    s = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="s")
    r = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="r")
    c = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="c")
    p = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="p")
    q = m.addMVar(shape=num_vars, vtype=GRB.CONTINUOUS, name="q")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    ## 各制約の違反度を最小化する
    m.setObjective(c1.T @ p + c2.T @ q, sense=gp.GRB.MINIMIZE)
    
    # 制約条件を設定
    ## 各係数行列のサイズを合わせる
    m.addConstr(As @ s + Ac_leq @ c + Ar_leq @ r + Ap @ p + Aq @ q <= b_leq, name="c_leq")
    #m.addConstr(Ac_eq @ c + Ar_eq @ r == b_eq, name="c_eq")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # パラメータ
    #m.Params.Presolve = 0
    #m.Params.Method = 0
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    """if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"車両が顧客{i}に到着する時刻は、{x[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')"""
    
    # モデルをテキストファイルにする
    m.write("out"+instance_name+".json")
        
    return m

## 双対問題を解く関数の定義

In [25]:
def solve_dual(As, Ap, b, c, instance_name, num_vars, PStarts, DStarts):
    import gurobipy as gp
    from gurobipy import GRB
    import numpy as np
    import time
    
    # インスタンスの生成
    m = gp.Model("LP_for_VRP" + instance_name)
    
    # 変数を設定
    """
    y_i : 主問題における制約式iの潜在価値
    """
    y = m.addMVar(shape=num_vars, lb=0.0, ub=float('inf'), vtype=GRB.CONTINUOUS, name="y")

    # モデルのアップデート
    m.update()
    
    # 目的関数を設定
    m.setObjective(-1 * b.T @ y, sense=gp.GRB.MAsIMIZE)
    
    # 制約条件を設定
    m.addConstr(As.T @ y >= 0, name="c1")
    m.addConstr(Ap.T @ y + c >= 0, name="c2")
    #m.addConstr(y >= 0, name="c3")

    # モデルのアップデート
    m.update()
    
    # 時間計測スタート
    start = time.time()
    
    # ホットスタートの使用
    for i, var in enumerate(m.getVars()):
        var.PStart = PStarts[i]
    for i, constr in enumerate(m.getConstrs()):
        #if i < DStarts.shape[0]:
        constr.DStart = DStarts[i]
    
    # パラメータの設定
    ##m.Params.Crossover = 4
    m.Params.Method = 0
    m.Params.Presolve = 0
    m.Params.Displayinterval = 2**31-1
    
    # 最適化
    m.optimize()
    
    # 時間計測ストップ
    elapsed_time = time.time() - start
    
    # 解の表示
    """if m.Status == gp.GRB.OPTIMAL:
        for i in range(num_vars):
            print(f"主問題における制約{i}の潜在価格は、{y[i].X}")
        print("最適値 : ", m.ObjVal)
    print('\033[34m'+f"実時間\t{elapsed_time}"+'\033[0m')"""
    
    # モデルをテキストファイルにする
    m.write("out"+instance_name+".mst")
        
    return m

## ①全体を1つのLPとして解くveb.

### 全体のPrimalを解く

In [26]:
As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2 = make_inputs(tour, Customers, Points, F, C)

Preceding constraints are done.
Time-window constarints are done.
Pick-up and delivery constarints are done.
Charge constarints are done.


In [ ]:
print("As : ", As.shape)
print("Ac_leq : ", Ac_leq.shape)
print("Ar_leq : ", Ar_leq.shape)
print("Ap : ", Ap.shape)
print("Aq : ", Aq.shape)
print("b_leq : ", b_leq.shape)

print("Ac_eq : ", Ac_eq.shape)
print("Ar_eq : ", Ar_eq.shape)
print("b_eq : ", b_eq.shape)

In [ ]:
b_leq

In [ ]:
for i in range(As.shape[0]):
    print(As[i], " ", Ac_leq[i], " ", Ar_leq[i], " ", Ap[i], " ", Aq[i])

In [27]:
# Gurobiによって最適解を求める
P = solve_primal(As, Ac_eq, Ac_leq, Ar_eq, Ar_leq, Ap, Aq, b_eq, b_leq, c1, c2, instance_name+"P", len(tour))

Using license file /Users/okamoto/gurobi.lic
Academic license - for non-commercial use only
Gurobi Optimizer version 9.0.2 build v9.0.2rc0 (mac64)
Optimize a model with 46 rows, 55 columns and 105 nonzeros
Model fingerprint: 0xba659d7c
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Presolve removed 26 rows and 38 columns
Presolve time: 0.01s
Presolved: 20 rows, 17 columns, 41 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.9199067e+02   5.738576e+00   0.000000e+00      0s
      17    2.0082444e+02   0.000000e+00   0.000000e+00      0s

Solved in 17 iterations and 0.03 seconds
Optimal objective  2.008244363e+02


In [ ]:
for var in P.getVars():
    print(var.varName, var.X)

In [ ]:
for constr in P.getConstrs():
    print(constr.Pi)

## ②前半と後半をつなげるveb.
1. 適当なところで前後に分ける
1. 前半後半それぞれのPrimalを解く
1. 前半と後半それぞれのPrimalの最適解を、全体のDualに入れて解く
1. 全体のPrimalの最適解を得る

### 1. 適当なところで前後に分ける
- ひとまず半分くらいで分けることにする

In [ ]:
threshold = len(tour)//2
#print(threshold)

former = tour[:threshold]
latter = tour[threshold:]
#print(f"巡回路全体は、{tour}")
print(f"前半は、{former}")
print(f"後半は、{latter}")

### 2. 前半後半それぞれのPrimalを解く

In [ ]:
import numpy as np
# 係数行列、ベクトルを整える
A1x, A1p, A2x, A2p, A3x, A3p, b1, b2, b3, c1, c2 = [], [], [], [], [], [], [], [], [], [], []
index = 0
index_P_f, index_P_l = [], []
## A1, A2, A3を定める
## b1, b2, b3を定める
for As_i, Ap_i, b_i in zip(As, Ap, b):
    if np.linalg.norm(As_i[threshold:], ord=2)==0.:
        A1x.append(As_i[:threshold])
        A1p.append(Ap_i[:threshold])
        b1.append(b_i)
        index_P_f.append(index)
    elif np.linalg.norm(As_i[:threshold], ord=2)==0.:
        A2x.append(As_i[threshold:])
        A2p.append(Ap_i[threshold:])
        b2.append(b_i)
        index_P_l.append(index)
    else:
        A3x.append(As_i)
        A3p.append(Ap_i)
        b3.append(b_i)
    index += 1
## c1, c2, c3を定める
c1 = c[:threshold]
c2 = c[threshold:]
## リストからnumpy arrayに変換
A1x = np.array(A1x)
A1p = np.array(A1p)
A2x = np.array(A2x)
A2p = np.array(A2p)
A3x = np.array(A3x)
A3p = np.array(A3p)
b1 = np.array(b1)
b2 = np.array(b2)
b3 = np.array(b3)
c1 = np.array(c1)
c2 = np.array(c2)
for i in range(1, 4):
    print(f"A{i}x : ", end="")
    print(eval("A"+str(i)+"x"))
    print(f"A{i}p : ", end="")
    print(eval("A"+str(i)+"p"))
    print(f"b{i} : ", end="")
    print(eval("b"+str(i)))
    if i <= 2:
        print(eval("c"+str(i)))

In [ ]:
# Gurobiによって最適解を求める
P_f = solve_primal(A1x, A1p, b1, c1, instance_name+"P_f", len(former))
P_l = solve_primal(A2x, A2p, b2, c2, instance_name+"P_l", len(latter))

In [ ]:
for v in P_f.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for v in P_l.getVars():
    print('%s %g %g' % (v.varName, v.x, v.VBasis))

In [ ]:
for constr in P_f.getConstrs()+P_l.getConstrs():
    print(constr.Pi, constr.CBasis)

### 3. 前半後半それぞれのPrimalの最適解を、全体のDualに入れて解く

In [ ]:
# 初期解の保存
PStarts = np.array([0 for constr in P_f.getConstrs()+P_l.getConstrs()])
y3 = np.zeros((As.shape[0]-PStarts.shape[0],))
PStarts = np.append(PStarts, y3)
for i, constr in zip(index_P_f, P_f.getConstrs()):
    PStarts[i] = constr.Pi
for i, constr in zip(index_P_l, P_l.getConstrs()):
    PStarts[i] = constr.Pi

DStarts = np.array([var.x for var in P_f.getVars()+P_l.getVars()])
#c3 = np.zeros((PStarts.shape[0],))
#DStarts = np.append(DStarts, c3)
"""for i, var in zip(index_P_f, P_f.getVars()):
    DStarts[i] = var.x
for i, var in zip(index_P_l, P_l.getVars()):
    DStarts[i] = var.x"""

In [ ]:
print(f"As.T.shape={As.T.shape}\t\t\tAp.T.shape={Ap.T.shape}")
for As_i, Ap_i in zip(As.T, Ap.T):
    print(As_i, "\t", Ap_i)

In [ ]:
import numpy as np
PStarts = np.array([-constr.Pi for constr in P.getConstrs()])
DStarts = np.array([var.x for var in P.getVars()])
#c3 = np.zeros((PStarts.shape[0],))
#DStarts = np.append(DStarts, c3)

In [ ]:
# Gurobiによって最適解を求める
D = solve_dual(As, Ap, b, c, instance_name+"D", PStarts.shape[0], PStarts, DStarts)

In [ ]:
c.T @ DStarts[len(DStarts)//2:]

In [ ]:
 -1 * b.T @ PStarts

In [ ]:
P.getObjective()

In [ ]:
D.getObjective()

In [ ]:
P.getObjective().getValue()

In [ ]:
D.getObjective().getValue()

In [ ]:
# 最適解
for var in D.getVars():
    print(var.varName, var.X)

### 4. 全体のPrimalの最適解を得る

In [ ]:
for constr in D.getConstrs():
    print(-constr.Pi)

In [ ]:
for var in P.getVars():
    print(var.varName, var.X)